# Chapitre 2 - Visualisation et Analyse descriptive des données départementales

In [ ]:
# Paramètre(s) du notebook

# VERBOSE=True
VERBOSE=False

OPTIONS=""
if not VERBOSE:
    OPTIONS="--quiet"


In [ ]:
def asciiprint(variable,desc):
    print("-"*100)
    print(variable,":",desc)
    print("-"*100)

In [ ]:
def showgraph(departements,df,dep_idf,df_idf,var,color,label):
    asciiprint(var,label)
    fig, ax = plt.subplots(figsize=(10,10))

    departements.plot(color='gray', ax=ax)
    df.plot(column=var, 
                        cmap=color, 
                        linewidth=0.5, 
                        edgecolor='black',
                        ax=ax, 
                        legend=True,
                        legend_kwds={'label': label, 'orientation': "horizontal"})
    ax.set_axis_off()
    
    fig, ax = plt.subplots(figsize=(10,10))

    dep_idf.plot(color='gray', ax=ax)
    df_idf.plot(column=var, 
                        cmap=color, 
                        linewidth=0.5, 
                        edgecolor='black',
                        ax=ax, 
                        legend=True,
                        legend_kwds={'label': label, 'orientation': "horizontal"})
    ax.set_axis_off()

In [ ]:
!pip install geopandas $OPTIONS
!pip install plotly $OPTIONS
!pip install seaborn $OPTIONS
!pip install missingno $OPTIONS
!pip install prince $OPTIONS

In [ ]:
#Datascience/Math manipulation libraries
import pandas as pd
import numpy as np

#Dataviz libraries
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import plotly.express as px
from IPython.core.display import display, HTML
import missingno as msno

# non supervised ML libraries
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import scipy.cluster.hierarchy as shc
from scipy.cluster.hierarchy import dendrogram,linkage
import prince

In [ ]:
DEPARTEMENTS_IDF=['75','77','78','91','92','93','94','95']

## 1) Chargement des données (Récup_données) et téléchargement de la localisation des départements français (restreint au champ métropolitain puis champ Ile de France)

In [ ]:
%store -r donnees_2018_hab
donnees_2018_hab

In [ ]:
%matplotlib inline
msno.matrix(donnees_2018_hab)
print("Il y a des données manquantes pour les crimes et délits pour les gendarmeries. Pour simplifier l'étude, nous n'imputons pas de valeurs explicites autres que 0 et travaillons sur le total gendarmerie+police nationale.")

In [ ]:
url_dep = 'https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb'
departements = gpd.read_file(url_dep)
departements = departements.set_index("code")
departements.drop(['nom'], axis=1, inplace = True)

In [ ]:
#Restriction au champ IDF pour une sous-carte
departements_idf = departements.loc[departements.index.isin(DEPARTEMENTS_IDF)]

In [ ]:
donnees_2018_hab_mini=donnees_2018_hab.drop(["D918","D118","REG","Crim_Del_GN_hab","Crim_Del_PN_hab","Crim_Del_PN_GN"],axis=1)
if VERBOSE:
    display(donnees_2018_hab_mini)

In [ ]:
if not departements_idf.empty and not donnees_2018_hab_mini.empty:
    print("1 - Récupération des données ====> OK ")
else:
    print("1 - Récupération des données ====> KO ")

## 2) Sélection des items que l'on souhaite représenter (au champ métropolitain puis IDF) 

In [ ]:
donnees_2018_hab.index.names = ['CODDEP']
donnees_2018_hab.describe()

In [ ]:
donnees_2018_hab_pauvrete = donnees_2018_hab.drop(['REG','MED18','D118','D918','RD18','T1_2018','Nb_PN_GN_dep_100k_hab','Crim_Del_PN_GN','Nb_Boucherie_dep_hab','Crim_Del_GN_hab','Crim_Del_PN_hab','Crim_Del_PN_GN_hab'], axis=1)
donnees_2018_hab_rev_median = donnees_2018_hab.drop(['REG','TP6018','D118','D918','RD18','T1_2018','Nb_PN_GN_dep_100k_hab','Crim_Del_PN_GN','Nb_Boucherie_dep_hab','Crim_Del_GN_hab','Crim_Del_PN_hab','Crim_Del_PN_GN_hab'], axis=1)
donnees_2018_hab_ratio_d1_d9 = donnees_2018_hab.drop(['REG','TP6018','D118','D918','MED18','T1_2018','Nb_PN_GN_dep_100k_hab','Crim_Del_PN_GN','Nb_Boucherie_dep_hab','Crim_Del_GN_hab','Crim_Del_PN_hab','Crim_Del_PN_GN_hab'], axis=1)
donnees_2018_hab_chomage = donnees_2018_hab.drop(['REG','TP6018','D118','D918','MED18','RD18','Nb_PN_GN_dep_100k_hab','Crim_Del_PN_GN','Nb_Boucherie_dep_hab','Crim_Del_GN_hab','Crim_Del_PN_hab','Crim_Del_PN_GN_hab'], axis=1)
donnees_2018_hab_boucherie = donnees_2018_hab.drop(['REG','TP6018','D118','D918','MED18','RD18','Nb_PN_GN_dep_100k_hab','Crim_Del_PN_GN','T1_2018','Crim_Del_GN_hab','Crim_Del_PN_hab','Crim_Del_PN_GN_hab'], axis=1)
donnees_2018_hab_nb_pn_gn = donnees_2018_hab.drop(['REG','TP6018','D118','D918','MED18','RD18','Nb_Boucherie_dep_hab','Crim_Del_PN_GN','T1_2018','Crim_Del_GN_hab','Crim_Del_PN_hab','Nb_Boucherie_dep_hab', 'Crim_Del_PN_GN_hab'], axis=1)
donnees_2018_hab_nb_crimes_delits = donnees_2018_hab.drop(['REG','TP6018','D118','D918','MED18','RD18','Nb_PN_GN_dep_100k_hab','Crim_Del_PN_GN','T1_2018','Crim_Del_GN_hab','Crim_Del_PN_hab','Nb_Boucherie_dep_hab'], axis=1)
donnees_2018_hab_crimes_delits = donnees_2018_hab.drop(['REG','TP6018','D118','D918','MED18','RD18','Nb_PN_GN_dep_100k_hab','Crim_Del_PN_GN','T1_2018','Crim_Del_GN_hab','Crim_Del_PN_hab','Nb_Boucherie_dep_hab'], axis=1)


In [ ]:
carto_pauvrete=departements.merge(donnees_2018_hab_pauvrete,left_index=True,right_index=True)
carto_rev_median=departements.merge(donnees_2018_hab_rev_median,left_index=True,right_index=True)
carto_ratio_d1_d9=departements.merge(donnees_2018_hab_ratio_d1_d9,left_index=True,right_index=True)
carto_chomage=departements.merge(donnees_2018_hab_chomage,left_index=True,right_index=True)
carto_boucherie=departements.merge(donnees_2018_hab_boucherie,left_index=True,right_index=True)
carto_nb_pn_gn=departements.merge(donnees_2018_hab_nb_pn_gn,left_index=True,right_index=True)
carto_crimes_delits=departements.merge(donnees_2018_hab_crimes_delits,left_index=True,right_index=True)

#Restriction au champ IDF pour une sous-carte

carto_pauvrete_idf = carto_pauvrete.loc[carto_pauvrete.index.isin(DEPARTEMENTS_IDF)]
carto_rev_median_idf = carto_rev_median.loc[carto_rev_median.index.isin(DEPARTEMENTS_IDF)]
carto_ratio_d1_d9_idf = carto_ratio_d1_d9.loc[carto_ratio_d1_d9.index.isin(DEPARTEMENTS_IDF)]
carto_chomage_idf = carto_chomage.loc[carto_chomage.index.isin(DEPARTEMENTS_IDF)]
carto_boucherie_idf = carto_boucherie.loc[carto_boucherie.index.isin(DEPARTEMENTS_IDF)]
carto_nb_pn_gn_idf = carto_nb_pn_gn.loc[carto_nb_pn_gn.index.isin(DEPARTEMENTS_IDF)]
carto_crimes_delits_idf = carto_crimes_delits.loc[carto_crimes_delits.index.isin(DEPARTEMENTS_IDF)]


In [ ]:
if not carto_crimes_delits_idf.empty:
    print("2 - Extractions des données ====> OK ")
else:
    print("2 - Extractions des données ====> KO ")

## 3) Représentation cartographique cloropèthe (au champ France Métropolitaine puis champ Ile de France) 

In [ ]:
print("3 - Visualisation géographique des départements en fonction des données : ")


* Taux de pauvreté monétaire

In [ ]:
showgraph(departements,carto_pauvrete,departements_idf,carto_pauvrete_idf,'TP6018','Purples','Taux de pauvreté monétaire (au seuil de 60% du niveau de vie médian)')

In [ ]:

print(" ----- COMMENTAIRE: En 2018, la pauvreté monétaire est plus marquée en Seine Saint-Denis, ainsi que dans les départements côtiers de l'Occitanie et sur la diagonale transfrontalière du Nord est de la France  ")

* Niveau de vie médian

In [ ]:
showgraph(departements,carto_rev_median,departements_idf,carto_rev_median_idf,'MED18','Oranges','Niveau de vie médian (en euros)')


In [ ]:
print(" ----- COMMENTAIRE : Le niveau de vie médian en euros en 2018 est supérieur en Haute-Savoie, département transfrontalier avec la Suisse et dans les départements de l'Ouest de l'Ile de France (Paris, Hauts-de-Seine, Yvelines). Le niveau de vie dans les départements ruraux de la diagonale du vide est moins élevé, tout comme celui de la région Hauts de France et de l'Occitanie.")

* Ratio inter-décile de niveau de vie 

In [ ]:
showgraph(departements,carto_ratio_d1_d9,departements_idf,carto_ratio_d1_d9_idf,'RD18','RdPu','Ratio inter-décile de niveau de vie')

In [ ]:
print(" ----- COMMENTAIRE : Le ratio inter-décile (D9/D1) de niveau de vie est particulièrement élevé à Paris, ainsi que dans les Hauts de Seine et dans une moindre mesure la Haute-Savoie. Dans la plupart des départements, le ratio inter-décile fluctue autour de 3.")

In [ ]:
asciiprint("T1_2018","Taux de chômage localisé")

* Taux de chômage localisé

In [ ]:
showgraph(departements,carto_chomage,departements_idf,carto_chomage_idf,'T1_2018','Greens','Taux de chômage localisé (en pourcentage)')

In [ ]:
print(" ----- COMMENTAIRE : La carte du taux de chômage localisé reflète celle du taux de pauvreté monétaire : la Seine Saint-Denis et les Pyrénnées Orientales ont un taux de chômage très élevé par rapport à la moyenne, et les départements des régions Hauts-de-France et d'Occitanie ont un taux de chômage élevé.")

* Nombre de boucheries pour 100k habitants

In [ ]:
showgraph(departements,carto_boucherie,departements_idf,carto_boucherie_idf,'Nb_Boucherie_dep_hab','Reds','Nombre de boucheries-charcuteries pour 100 000 habitants')


In [ ]:
print(" ----- COMMENTAIRE : Le nombre de boucheries-charcuteries pour 100 000 habitants est particulièrement supérieur en Corse-du-Sud et en Lozère ainsi que dans le Cantal. Paris et la Seine Saint Denis paraissent sureprésentés graphiquement, mais l'échelle n'est pas la même pour la région Ile de France. Ces départements se situent dans la moyenne des autres départements français.")

* Nombre de policiers et gendarmes (2019) pour 100k habitants

In [ ]:
showgraph(departements,carto_nb_pn_gn,departements_idf,carto_nb_pn_gn_idf,'Nb_PN_GN_dep_100k_hab','YlOrBr','Nombre de policiers et gendarmes (2019) pour 100 000 habitants')

In [ ]:
print(" ----- COMMENTAIRE : Le nombre de policiers et gendarmes pour 100 000 habitants est particulièrement élevé à Paris par rapport aux départements de la petite couronne, ce qui s'explique en partie par la densité de population et la centralisation administrative dans la capitale. Par ailleurs, la Haute-Marne et la Corse-du-Sud apparaissent surreprésentées.") 

* Nombre de crimes et délits répertoriés pour 100k habitants

In [ ]:
showgraph(departements,carto_crimes_delits,departements_idf,carto_crimes_delits_idf,'Crim_Del_PN_GN_hab','YlGnBu','Nombre de crimes et délits répertoriés pour 100 000 habitants')

In [ ]:
print(" ----- COMMENTAIRE : Graphiquement, on constate que le nombre de crimes et délits rapportés est plus important dans les grandes agglomérations (Paris, Lyon/Rhône, Marseille/Bouches du Rhône, Bordeaux/Gironde, Nantes/Loire-Atlantique), et dans les départements côtiers (le long de la Méditérrannée). La Corse se dinstingue par un plus faible nombre de crimes et délits rapportés.") 

# 4) Analyse univariée 

In [ ]:
print('*'*100)
print('*'*100)
print("4 - Analyse univariée avec un diagramme en boîte pour chaque variable explicative")

In [ ]:
if VERBOSE:
    display(donnees_2018_hab_mini.info())

In [ ]:
fig = px.box(donnees_2018_hab_mini, y="TP6018",hover_data=[ 'Libellé'], labels=dict(TP6018 = 'Taux de pauvreté monétaire au seuil de 60%'))

fig.write_html('figure_TP6018.html', auto_open=True)
display(HTML("figure_TP6018.html"))

In [ ]:
fig2 = px.box(donnees_2018_hab_mini, y="MED18",hover_data=[ 'Libellé'],labels=dict(MED18 = 'Niveau de vie médian'))

fig2.write_html('figure_MED18.html', auto_open=True)
display(HTML("figure_MED18.html"))

In [ ]:
fig3= px.box(donnees_2018_hab_mini, y="RD18",hover_data=[ 'Libellé'], labels=dict(RD18 = 'Ratio inter-décile'))

fig3.write_html('figure_RD_2018.html', auto_open=True)
display(HTML("figure_RD_2018.html"))

In [ ]:
fig4 = px.box(donnees_2018_hab_mini, y="T1_2018",hover_data=[ 'Libellé'], labels=dict(T1_2018 = 'Taux de chômage'))

fig4.write_html('figure_T1_2018.html', auto_open=True)
display(HTML("figure_T1_2018.html"))

In [ ]:
fig5 = px.box(donnees_2018_hab_mini, y="Nb_Boucherie_dep_hab",hover_data=[ 'Libellé'], labels=dict(Nb_Boucherie_dep_hab = 'Nombre de boucheries pour 100 000 habitants'))

fig5.write_html('figure_Nb_Boucherie_dep_hab.html', auto_open=True)
display(HTML("figure_Nb_Boucherie_dep_hab.html"))


In [ ]:
fig6 = px.box(donnees_2018_hab_mini, y="Nb_PN_GN_dep_100k_hab",hover_data=[ 'Libellé'], labels=dict(Nb_PN_GN_dep_100k_hab = 'Nombre de policiers et gendarmes pour 100 000 habitants'))

fig6.write_html('figure_Nb_PN_GN_dep_100k_hab.html', auto_open=True)
display(HTML("figure_Nb_PN_GN_dep_100k_hab.html"))

In [ ]:
fig7 = px.box(donnees_2018_hab_mini, y="Crim_Del_PN_GN_hab",hover_data=[ 'Libellé'], labels=dict(Crim_Del_PN_GN_hab= 'Nombre de crimes et délits reportés pour 100 000 habitants'))

fig7.write_html('figure_Crim_Del_PN_GN_hab.html', auto_open=True)
display(HTML("figure_Crim_Del_PN_GN_hab.html"))

# 5) Analyse bivariée 

In [ ]:
print('*'*100)
print('*'*100)
print("5 - Analyse bivariée : matrice de corrélation des variables explicatives ")

In [ ]:
plt.figure(figsize=(16, 6))
mask = np.triu(np.ones_like(donnees_2018_hab_mini.corr(), dtype=np.bool_))
heatmap = sns.heatmap(donnees_2018_hab_mini.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')

In [ ]:
print(" ----- COMMENTAIRE : Les seules corrélations négatives sont reliées au niveau de vie médian et concernent les variables de taux de chômage, de taux de pauvreté et dans une moindre mesure du nombre de boucheries-charcuteries. On remarque également une très faible corrélation négative entre le nombre de crimes et délits rapportés avec le nombre de boucheries charcuteries. Toutes les autres corrélations sont positives et attendues : lien entre la pauvreté et le chômage, ou la criminalité et le nombre de policiers (chiffre noir de la délinquance), ou bien encore le ratio inter-décile et la délinquance (reflétant les inégalités de revenus, donc potentiellement les tensions sociales).")

In [ ]:
sns.pairplot(donnees_2018_hab_mini[['TP6018','MED18', 'RD18', 'T1_2018', 'Nb_Boucherie_dep_hab', 'Nb_PN_GN_dep_100k_hab', 'Crim_Del_PN_GN_hab']])

In [ ]:
print(" ----- COMMENTAIRE : On constate une forte relation linéaire entre le taux de chômage et le taux de pauvreté. Au contraire, le lien entre le nombre de crimes et délits rapportés et nos variables explicatives semble plus limité (fort regroupement de valeurs sur l'axe des ordonnées malgré une forte dispersion sur l'axe des abscisses).") 

print(" ----- Cette relation est approfondie dans les graphiques ci-dessous intégrant la droite de régression et l'intervalle de confiance de notre estimateur.")


In [ ]:
sns.lmplot(x="Nb_Boucherie_dep_hab", y='Crim_Del_PN_GN_hab', data=donnees_2018_hab_mini)
plt.title('Crimes et délits rapportés en fonction du nombre de boucheries pour 100 000 habitants')

sns.lmplot(x="TP6018", y='Crim_Del_PN_GN_hab', data=donnees_2018_hab_mini)
plt.title('Crimes et délits rapportés en fonction du taux de pauvreté monétaire')

sns.lmplot(x="RD18", y='Crim_Del_PN_GN_hab', data=donnees_2018_hab_mini)
plt.title('Crimes et délits rapportés en fonction du ratio inter-décile de niveau de vie')

sns.lmplot(x="MED18", y='Crim_Del_PN_GN_hab', data=donnees_2018_hab_mini)
plt.title('Crimes et délits rapportés en fonction du niveau de vie médian')

sns.lmplot(x="T1_2018", y='Crim_Del_PN_GN_hab', data=donnees_2018_hab_mini)
plt.title('Crimes et délits rapportés en fonction du taux de chômage')

sns.lmplot(x="Nb_PN_GN_dep_100k_hab", y='Crim_Del_PN_GN_hab', data=donnees_2018_hab_mini)
plt.title('Crimes et délits rapportés en fonction du nombre de policiers et gendarmes pour 100 000 habitants')


# 6) Analyse multivariée

In [ ]:
print('*'*100)
print('*'*100)
print("6 - Analyse multivariée : Analyses en Composantes Principales (ACP) et clustering par la méthode de Ward")

In [ ]:
X=donnees_2018_hab_mini[['TP6018','MED18', 'RD18', 'T1_2018', 'Nb_Boucherie_dep_hab', 'Nb_PN_GN_dep_100k_hab', 'Crim_Del_PN_GN_hab']]

In [ ]:
pca = prince.PCA(
     n_components=10,
     n_iter=3,
     rescale_with_mean=True,
     rescale_with_std=True,
     copy=True,
     check_input=True,
     engine='auto',
     random_state=42 )
pca = pca.fit(X)

In [ ]:
pca.eigenvalues_

In [ ]:
pca.explained_inertia_

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1, 2, 1)
plt.plot(pca.eigenvalues_,marker='*')
plt.title('Valeurs Propres')

plt.subplot(1, 2, 2)
plt.plot(np.cumsum(pca.explained_inertia_),marker='*')
plt.title('Inertie cumulée')

plt.show()

In [ ]:
print(" ----- COMMENTAIRE : On cherche les composantes principales de telle sorte à réduire le nombre de variables initiales qui sont très corrélées.") 

print(" ----Par le critère du coude sur les valeurs propres associées à chaque variable, on choisit de retenir 3 composantes principales.")

print("----Le critère d'inertie cumulée est satisfaisant pour 2 ou 3 composantes princiaples.") 

In [ ]:
resPCA=pca.transform(X)

In [ ]:
print(" ----- COMMENTAIRE : On cherche désormais à regrouper les départements par cluster pour faire apparaître des caractéristiques communes.") 

In [ ]:
inertias = []

# Creating 10 K-Mean models while varying the number of clusters (k)
for k in range(1,10):
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(resPCA.iloc[:,:3])
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(range(1,10), inertias, '-p', color='gold')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
#plt.xticks(ks)
plt.show()

In [ ]:
print(" ----- COMMENTAIRE : L'inertie est décroissante par rapport au nombre de clusters car on cherche à réduire l'inertie intra-classe. Le critère du coude suggère 3 ou 5 clusters. Pour une meilleure précision, on choisit au départ 5 clusters puis on applique la méthode de Ward.") 

In [ ]:
plt.figure(figsize=(20, 5))
plt.title("Customer Dendograms")
dend = shc.dendrogram(shc.linkage(resPCA, method='ward'))

In [ ]:
model = KMeans(n_clusters=3)
model.fit(resPCA.iloc[:,:2])
labels = model.predict(resPCA.iloc[:,:2])
plt.scatter(resPCA[0], resPCA[1], c=labels)
plt.show()

In [ ]:
print(" ----- COMMENTAIRE : La méthode de Ward indique 3 clusters.") 

In [ ]:
resKM=KMeans(n_clusters=3)
resKM.fit(resPCA)

In [ ]:
resKM

In [ ]:
X["Cluster"]=resKM.labels_

In [ ]:
X.groupby("Cluster").mean()[['TP6018','MED18', 'RD18', 'T1_2018', 'Nb_Boucherie_dep_hab', 'Nb_PN_GN_dep_100k_hab', 'Crim_Del_PN_GN_hab']].round(2).transpose()

In [ ]:
X.groupby("Cluster").describe()

In [ ]:
sns.scatterplot(x="Nb_Boucherie_dep_hab",y="Crim_Del_PN_GN_hab",data=X,hue="Cluster")

In [ ]:
print(" ----- COMMENTAIRE : On distingue 3 cluster ")

print(" ----- Le cluster 0 avec un faible taux de pauvreté (en moyenne 13%), un taux de chômage moyen, un faible nombre de boucheries charcuterie et un nombre de crimes et délits rapportés inférieur à la moyenne.") 

print(" ----- Le cluster 1 se caractérise par une pauvreté plus élevée (en moyenne 18%), une délinquance, un nombre de boucherie charcuteries et un chômage plus élevés")

print(" ----- Le cluster 2 représente la spécificité de Paris qui se distingue par un fort ratio inter décile et un nombre de crimes et délits rapportés bien plus élevés")

In [ ]:
X_cluster = X.drop(['TP6018','MED18', 'RD18', 'T1_2018', 'Nb_Boucherie_dep_hab', 'Nb_PN_GN_dep_100k_hab', 'Crim_Del_PN_GN_hab'], axis=1)
X_cluster.astype(int)

In [ ]:
carto_cluster=departements.merge(X_cluster,left_index=True,right_index=True)
carto_cluster

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

departements.plot(color='gray', ax=ax)
carto_cluster.plot(column='Cluster', 
                    cmap='Reds', 
                    linewidth=0.5, 
                    edgecolor='black',
                    ax=ax, 
                    legend=True,
                    legend_kwds={'label': 'Cluster', 'orientation': "horizontal"})
ax.set_axis_off()

In [ ]:
print(" ------ COMMENTAIRE : Le cluster 1 se concentre le long de la côte Méditérannée et dans certains départements de la diagonale du vide, ainsi qu'en Seine Saint-Denis, le Val d'Oise, et le Nord de la France. Paris se distingue enfin par sa spécificité.")

In [ ]:
%store X_cluster